<a href="https://colab.research.google.com/github/Patisson4/ML/blob/lab-5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [1]:
import pandas as pd
import nltk

data = pd.read_csv(
    'https://github.com/sismetanin/rureviews/blob/master/women-clothing-accessories.3-class.balanced.csv?raw=true',
    sep='\t')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdpol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdpol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import string
from time import perf_counter
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

In [3]:
data['format'] = data['review']

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
start = perf_counter()

analyzer = MorphAnalyzer()

for i in range(len(data)):
    for pun in string.punctuation:
        data.format[i] = data.format[i].replace(pun, "")
    tokens = [analyzer.parse(word.lower())[0].normal_form for word in word_tokenize(data.format[i])]
    data.format[i] = " ".join(tokens)

print(data.head(10))
print(f'Performed in {perf_counter() - start}')

In [9]:
results = []
X_train, X_test, y_train, y_test = train_test_split(data.format, data.sentiment, train_size=0.7)

def naive_bayes_classifier(vectorizer):
    started = perf_counter()

    classifier = MultinomialNB()
    classifier.fit(vectorizer.fit_transform(X_train), y_train)

    predicted = classifier.predict(vectorizer.transform(X_test))
    results.append([classification_report(y_test, predicted, output_dict=True, zero_division=0), vectorizer])

    print(f"ngram_range: {vectorizer.ngram_range}", f"analyzer: {vectorizer.analyzer}", classification_report(y_test, predicted, output_dict=False, zero_division=0), sep='\n')
    print(f"Performed in {perf_counter() - started}")

In [10]:
MIN_NGRAM = 2
MAX_NGRAM = 8

for min_n in range(MIN_NGRAM, MAX_NGRAM):
    for max_n in range(min_n, MAX_NGRAM):
        naive_bayes_classifier(CountVectorizer(ngram_range=(min_n, max_n), stop_words=stopwords.words('russian')))

ngram_range: (2, 2)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.58      0.52      0.55      9006
    negative       0.63      0.68      0.65      8980
    positive       0.77      0.80      0.78      9015

    accuracy                           0.66     27001
   macro avg       0.66      0.66      0.66     27001
weighted avg       0.66      0.66      0.66     27001

Performed in 9.276157999993302
ngram_range: (2, 3)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.58      0.50      0.54      9006
    negative       0.63      0.68      0.65      8980
    positive       0.76      0.80      0.78      9015

    accuracy                           0.66     27001
   macro avg       0.65      0.66      0.66     27001
weighted avg       0.65      0.66      0.66     27001

Performed in 7.881116900010966
ngram_range: (2, 4)
analyzer: word
              precision    recall  f1-score   support

    neautral    

In [11]:
MIN_NGRAM = 2
MAX_NGRAM = 4

for min_n in range(MIN_NGRAM, MAX_NGRAM):
    for max_n in range(min_n, MAX_NGRAM):
        for max_df in [0.1, 0.5]:
            for min_df in [0, 0.001, 0.01]:
                for max_features in [1000, 25000, 50000]:
                    print(f"min_df: {min_df}; max_df: {max_df}")
                    naive_bayes_classifier(TfidfVectorizer(ngram_range=(min_n, max_n), max_df=max_df, min_df=min_df, max_features=max_features, stop_words=stopwords.words('russian')))

min_df: 0; max_df: 0.1
ngram_range: (2, 2)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.53      0.43      0.48      9006
    negative       0.51      0.69      0.58      8980
    positive       0.74      0.63      0.68      9015

    accuracy                           0.58     27001
   macro avg       0.60      0.58      0.58     27001
weighted avg       0.60      0.58      0.58     27001

Performed in 7.670804399996996
min_df: 0; max_df: 0.1
ngram_range: (2, 2)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.57      0.55      0.56      9006
    negative       0.61      0.66      0.63      8980
    positive       0.78      0.76      0.77      9015

    accuracy                           0.65     27001
   macro avg       0.66      0.65      0.65     27001
weighted avg       0.66      0.65      0.65     27001

Performed in 7.378487099980703
min_df: 0; max_df: 0.1
ngram_range: (2, 2)
analyzer: word
  

C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.23      0.01      0.01      9006
    negative       0.33      0.98      0.49      8980
    positive       0.00      0.00      0.00      9015

    accuracy                           0.33     27001
   macro avg       0.19      0.33      0.17     27001
weighted avg       0.19      0.33      0.17     27001

Performed in 6.230096399987815
min_df: 0.01; max_df: 0.1


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.23      0.01      0.01      9006
    negative       0.33      0.98      0.49      8980
    positive       0.00      0.00      0.00      9015

    accuracy                           0.33     27001
   macro avg       0.19      0.33      0.17     27001
weighted avg       0.19      0.33      0.17     27001

Performed in 6.977910599991446
min_df: 0.01; max_df: 0.1


...

C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.23      0.01      0.01      9006
    negative       0.33      0.98      0.49      8980
    positive       0.00      0.00      0.00      9015

    accuracy                           0.33     27001
   macro avg       0.19      0.33      0.17     27001
weighted avg       0.19      0.33      0.17     27001

Performed in 6.668638300005114
min_df: 0.01; max_df: 0.5


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

ngram_range: (3, 3)
analyzer: word
              precision    recall  f1-score   support

    neautral       0.23      0.01      0.01      9006
    negative       0.33      0.98      0.49      8980
    positive       0.00      0.00      0.00      9015

    accuracy                           0.33     27001
   macro avg       0.19      0.33      0.17     27001
weighted avg       0.19      0.33      0.17     27001

Performed in 6.1510020999994595


C:\Users\mdpol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
MIN_NGRAM = 2
MAX_NGRAM = 8

for min_n in range(MIN_NGRAM, MAX_NGRAM):
    for max_n in range(min_n, MAX_NGRAM):
        naive_bayes_classifier(CountVectorizer(ngram_range=(min_n, max_n), analyzer='char', stop_words=stopwords.words('russian')))

ngram_range: (2, 2)
analyzer: char
              precision    recall  f1-score   support

    neautral       0.52      0.61      0.56      9006
    negative       0.67      0.57      0.61      8980
    positive       0.76      0.73      0.74      9015

    accuracy                           0.64     27001
   macro avg       0.65      0.64      0.64     27001
weighted avg       0.65      0.64      0.64     27001

Performed in 7.335666800005129
ngram_range: (2, 3)
analyzer: char
              precision    recall  f1-score   support

    neautral       0.55      0.67      0.60      9006
    negative       0.70      0.58      0.64      8980
    positive       0.82      0.78      0.80      9015

    accuracy                           0.68     27001
   macro avg       0.69      0.68      0.68     27001
weighted avg       0.69      0.68      0.68     27001

Performed in 16.22408459999133
ngram_range: (2, 4)
analyzer: char
              precision    recall  f1-score   support

    neautral    

In [ ]:
raw_data = []
for result, vectorizer in results:
    score = result["weighted avg"]
    params = []
    if isinstance(vectorizer, TfidfVectorizer):
        params.append(f"min_df: {vectorizer.min_df} max_df: {vectorizer.max_df} max_features: {vectorizer.max_features}")
    params.append(f"ngram: {vectorizer.ngram_range}")
    raw_data.append({"Vectorizer": str(type(vectorizer)).split('.')[-1], "Analyzer": vectorizer.analyzer, "Parameters": params, "Precision": score["precision"], "Recall": score["recall"], "F1-Score": score["f1-score"]})
data_result = pd.DataFrame(raw_data)
data_result.sort_values(by=["F1-Score"], ascending=False).head(10)

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b\w{2}', "Mauris id augue ac risus convallis laoreet a eu tellus")
print(result)

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('\.',
                  'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque tincidunt ac nulla sed pellentesque. Fusce non pellentesque elit, ac viverra est. Maecenas laoreet, leo eget venenatis dictum, nisi eros vestibulum sem, nec tristique est risus et lorem. Cras faucibus lorem ante, sed volutpat est hendrerit fermentum. Vestibulum aliquam non lectus eu blandit. Nam suscipit lacus id sollicitudin aliquam. Etiam ut ultricies mauris.',
                  maxsplit=2)
print(result)

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print(result)

**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub(r'\d', 'DIG', '1 one 2 two 3 three 4 four 10 ten')
print(result)

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub(r'\w+://\S+', '[link removed]',
                'Docs: https://docs.python.org/3/library/re.html Meme: https://www.youtube.com/watch?v=dQw4w9WgXcQ&t=0s')
print(result)

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
rg = re.compile('\w{4,}')
rg.findall("Vestibulum pellentesque facilisis orci, id convallis dui vulputate id.")

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
re.findall(r'@[\w.]+', 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')